In [1]:
import os
import csv
import praw
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timezone

In [2]:
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
user_agent = os.getenv('USER_AGENT')
user_name = os.getenv('USER_NAME')

In [3]:
reddit = praw.Reddit(
  client_id=client_id,
  client_secret=client_secret,
  user_agent=user_agent
)

In [4]:
user = reddit.redditor(user_name)

In [14]:
posts = []

for submission in user.submissions.new(limit=2000):
  if submission.is_self and submission.selftext:
    post_text = submission.selftext
  else:
    post_text = None
  posts.append({
    'author': submission.author,
    'title': submission.title,
    'score': submission.score,
    'num_comments': submission.num_comments,
    'created_utc': datetime.fromtimestamp(submission.created_utc, tz=timezone.utc),
    'post_text': post_text,
    'url': submission.url,
    'upvote_ratio': submission.upvote_ratio
  })

In [15]:
df = pd.DataFrame(posts)

df.head()

,author,title,score,num_comments,created_utc,post_text,url,upvote_ratio
0,reddit,Is this really from Reddit? How to tell:,67,70,2024-02-22 16:59:37+00:00,None,/r/help/comments/1awsj1k/is_this_really_from_r...,0.91
1,reddit,Is this really from Reddit? How to tell:,56,1,2024-02-22 15:54:53+00:00,None,/r/help/comments/1awsj1k/is_this_really_from_r...,0.87
2,reddit,Is this really from Reddit? How to tell:,89,62,2024-02-22 00:33:52+00:00,[removed],https://www.reddit.com/r/help/comments/1awsj1k...,0.90
3,reddit,Join our CEO tomorrow to discuss the API,0,0,2023-06-08 18:54:58+00:00,"Reddit CEO, u/spez, will be here tomorrow to h...",https://www.reddit.com/r/reddit/comments/144ho...,0.14
4,reddit,Sat 2021-12-11,643,336,2021-12-13 00:02:38+00:00,accessing vault............unlocked\r\n\r\ncal...,https://www.reddit.com/r/nameaserver/comments/...,0.98


In [12]:
keys = posts[0].keys()
with open('reddit.csv', 'w', newline='', encoding='utf-8') as file:
  writer = csv.DictWriter(file, fieldnames=keys)
  writer.writeheader()
  writer.writerows(posts)